- Run Zookeeper
    - .\bin\windows\zookeeper-server-start.bat .\config\zookeeper.properties

- Start Kafka Server
    - .\bin\windows\kafka-server-start.bat .\config\server.properties

- Create Topics 
    - .\bin\windows\kafka-topics.bat --create --topic test --bootstrap-server localhost:9092

- Consume Topics 
    - .\bin\windows\kafka-console-consumer.bat --topic test --from-beginning --bootstrap-server localhost:9092


In [5]:
import pandas as pd
from kafka import KafkaConsumer, KafkaProducer
from time import sleep
from json import dumps
import json
from kafka import KafkaProducer
from json import dumps



In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
from nltk.corpus import stopwords

import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import LatentDirichletAllocation

from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score


In [7]:

producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: dumps(x).encode('utf-8'))


In [8]:
#producer.send('demo_test', value="{'name': 'Consumer is was working well'}")

In [9]:
#df = pd.read_csv("indexProcessed.csv")
IMDB_df = pd.read_csv('https://raw.githubusercontent.com/manojjha/BITS_Assignments/main/NLP/IMDB%20Dataset.csv?token=GHSAT0AAAAAACJXQMXDNSKYL6LUDAUK5SISZMICSXQ')
IMDB_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
'''while True:
    dict_stock = df.sample(1).to_dict(orient="records")[0]
    producer.send('test', value=dict_stock)
    '''

'while True:\n    dict_stock = df.sample(1).to_dict(orient="records")[0]\n    producer.send(\'test\', value=dict_stock)\n    '

In [11]:
IMDB_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [12]:
import nltk
nltk.download('stopwords')
tokenizer = ToktokTokenizer()

stopword_list = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manoj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
#Removing noise text

def strip_html(text):
  soup = BeautifulSoup(text, 'html.parser')
  return soup.get_text()

def remove_between_square_brackets(text):
  return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
  text = strip_html(text)
  text = remove_between_square_brackets(text)
  return text

IMDB_df['review'] = IMDB_df['review'].apply(denoise_text)

C:\Users\manoj\AppData\Local\Temp\ipykernel_1288\3752247428.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [14]:
#remove special characters

def remove_special_characters(text, remove_digits=True):
  pattern = r'[^a-zA-Z0-9\s]'
  text = re.sub(pattern,'', text)
  return text

IMDB_df['review'] = IMDB_df['review'].apply(remove_special_characters)

In [15]:
# Removing stopwords

stop = set(stopwords.words('english'))
print(stop)

{'but', 'are', 've', "she's", 'just', 'itself', 'than', 'whom', "that'll", 'ours', 'then', 'y', 'had', "needn't", 'as', 'won', 'more', 'at', 'aren', 'her', 'until', 'them', 'wouldn', 'he', 'd', 'an', 'myself', 'hadn', 'his', 'some', "wouldn't", 'him', 'me', 'nor', "isn't", "won't", 'hers', 'was', "you'll", 'who', "you're", 'the', 'no', "mustn't", "aren't", 'not', 'should', 'ain', 'against', 'through', 'herself', 'for', 'so', 'will', 'here', 'couldn', 'wasn', 'where', "didn't", 'other', 'between', 'it', 'that', 'did', "shan't", "wasn't", "weren't", 'we', 'why', 'yourselves', 'doesn', 'does', 'when', 'below', 'how', 'their', 'been', "you'd", 'll', 'few', 'hasn', 'weren', 'don', 'down', 'am', 'up', 'themselves', 'now', 'm', 'on', 'those', 'what', 'ourselves', 't', 'over', 'same', 'o', 're', 'my', "shouldn't", 'has', 'do', 'again', 'shan', 'very', 'have', 'by', 'these', 'own', "doesn't", 'ma', 'both', 'into', 'during', 'shouldn', 'which', 'doing', 'to', 'while', 'our', 'isn', 'its', 'under

In [16]:
def remove_stopwords(text, is_lower_case = False):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  if is_lower_case:
    filtered_tokens = [token for token in tokens if token not in stopword_list ]

  else:
    filtered_tokens = [token for token in tokens if token.lower() not in stopword_list ]

  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

IMDB_df['review'] = IMDB_df['review'].apply(remove_stopwords)

In [17]:
IMDB_df.head()

,review,sentiment
0,One reviewers mentioned watching 1 Oz episode ...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,Basically theres family little boy Jake thinks...,negative
4,Petter Matteis Love Time Money visually stunni...,positive


In [18]:
# text stemming

def simple_stemmer(text):
  ps = nltk.porter.PorterStemmer()
  text = ' '.join([ps.stem(word) for word in text.split()])
  return text

IMDB_df['review'] = IMDB_df['review'].apply(simple_stemmer)

In [19]:
IMDB_df['cleaned_review'] = IMDB_df['review']
IMDB_df.head()

,review,sentiment,cleaned_review
0,one review mention watch 1 oz episod youll hoo...,positive,one review mention watch 1 oz episod youll hoo...
1,wonder littl product film techniqu unassum old...,positive,wonder littl product film techniqu unassum old...
2,thought wonder way spend time hot summer weeke...,positive,thought wonder way spend time hot summer weeke...
3,basic there famili littl boy jake think there ...,negative,basic there famili littl boy jake think there ...
4,petter mattei love time money visual stun film...,positive,petter mattei love time money visual stun film...


In [20]:
#  Create X and y objects
X = IMDB_df['cleaned_review']
y = IMDB_df['sentiment']

In [21]:
label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(y)

print(y_encoded)

[1 1 1 ... 0 0 0]


In [22]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

X_tfidf = tfidf_vectorizer.fit_transform(X)

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, y_encoded, test_size=0.2, random_state=42)

#### Train Support Vector machine model

In [23]:
# Initialize and train the SVM model
svm_tfidf = SVC(kernel='linear', random_state=42)
svm_tfidf.fit(X_train_tfidf, y_train_tfidf)

y_pred_tfidf = svm_tfidf.predict(X_test_tfidf)



In [25]:
conf_matrix_tfidf = confusion_matrix(y_test_tfidf, y_pred_tfidf)
print("Confusion Matrix (TFIDF):")
print(conf_matrix_tfidf)

accuracy = svm_tfidf.score(X_train_tfidf, y_train_tfidf)
print("Training accuracy:", accuracy * 100, "%")


Confusion Matrix (TFIDF):
[[4312  649]
 [ 509 4530]]
Training accuracy: 91.9625 %


In [29]:
print(IMDB_df['review'].to_dict)

<bound method Series.to_dict of 0        one review mention watch 1 oz episod youll hoo...
1        wonder littl product film techniqu unassum old...
2        thought wonder way spend time hot summer weeke...
3        basic there famili littl boy jake think there ...
4        petter mattei love time money visual stun film...
                               ...                        
49995    thought movi right good job wasnt creativ orig...
49996    bad plot bad dialogu bad act idiot direct anno...
49997    cathol taught parochi elementari school nun ta...
49998    im go disagre previou comment side maltin one ...
49999    one expect star trek movi high art fan expect ...
Name: review, Length: 50000, dtype: object>


### Batch processing using IMDB dataframe data

In [31]:
import time

for index, row in IMDB_df.head(10).iterrows():
    review_text = row['review']
    cleaned_text = simple_stemmer(review_text)
    
    new_text_tfidf = tfidf_vectorizer.transform([cleaned_text])
    
    prediction = svm_tfidf.predict(new_text_tfidf)
    
    predicted_sentiment = label_encoder.inverse_transform(prediction)[0]
    
    message = {
        'review': review_text,
        'predicted_sentiment': predicted_sentiment
    }

    producer.send('test', value=message)
    time.sleep(5)


### Stream processing from live youtube chat comments

In [32]:
import pytchat

chat = pytchat.create(video_id = "sHwxI4FX6Jw")

while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.message}")
        
    review_text = c.message
    print(review_text)
    cleaned_text = simple_stemmer(review_text)
    
    new_text_tfidf = tfidf_vectorizer.transform([cleaned_text])
    
    prediction = svm_tfidf.predict(new_text_tfidf)
    
    predicted_sentiment = label_encoder.inverse_transform(prediction)[0]
    
    message = {
        'review': review_text,
        'predicted_sentiment': predicted_sentiment
    }

    producer.send('test', value=message)
    time.sleep(5)

THANKS JÜRGEN
HUSAM WAS KLOPP OUT IM SURE HIS HAPPY
Xabi appointed too early
Husam Raid
HUSAM RAID
We'll be just fine inshaallah
Housam raid
He knew he couldn't compete with Pep:loudly_crying_face::loudly_crying_face::loudly_crying_face:
Lil Tom wanted Klopp gone..Careful what you bleat for:grinning_face_with_sweat::grinning_face_with_sweat:
husam raid
Husam is Finished
LIVERPOOL TAKING MASSIVE STEPBACK KLOPP SAW THE WRITING ON THE WALL
@Markyy he won once, and goes first this season
He knew he cant win a cheat
Husam raid
HUSAM RAID
THANK U JURGEN
@Markyy only guy who has competed with him..Mickey Mouse League
Husam is Done and he knows it
Xabi Alonso :waving_hand::waving_hand::waving_hand::waving_hand::waving_hand:
No matter how good he is, the cheater always wins
KLOPPER TAKING A YEAR OFF THEN COACHING IN GERMANY 
Husam FRAUD raid!
Husam turning into STaffy next season
Salad will be out next
zee zoo
potter in
@Drunk Viggo dont be dis respectful u haywan
jesus wept hurry up
Jürgen Klo